# Interactive peak search
Try running it in a live notebook for animation!

In [1]:
# Reload modules every time code is called. Set autoreload 0 to disable
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

import lightlab.util.sweep as sUtil
from lightlab.util.io import RangeError

IndentationError: expected an indented block (sweep.py, line 1308)

In [ ]:
livePlots = True

In [ ]:
center = .82
amp = .7
fwhm = .2

defaultNoise = amp * 5e-3
noise = defaultNoise

assertionTolerance = .2

def myPeakedFun(x):
    y = amp / (1 + (2 * (x - center) / fwhm) ** 2) + noise * np.random.randn()
    return y
xq = np.linspace(0,3, 10)
plt.plot(xq, myPeakedFun(xq))
plt.title('Poor, low-res sampling of underlying peak')

In [ ]:
for noi, nSwarm in zip([defaultNoise, 5e-2], [3, 7]):
    noise = noi
    xPeak, yPeak = sUtil.peakSearch(evalPointFun=myPeakedFun, startBounds=[0,3],
                             nSwarm=nSwarm, xTol=.01, yTol=amp/100, livePlot=livePlots)
    assert abs(xPeak - center) < assertionTolerance
    assert abs(yPeak - amp) < assertionTolerance
noise = defaultNoise

## Interactive peak descent through binary search

In [ ]:
binSearchOpts = dict(evalPointFun=myPeakedFun, xTol=.005, livePlot=livePlots)

### This is easy, well bounded

In [ ]:
rightBounds = [xPeak, 3]
leftBounds = [0, xPeak]
hwhmKwargs = dict(targetY=0.5*yPeak, **binSearchOpts)
xRightHalf = sUtil.binarySearch(startBounds=rightBounds, **hwhmKwargs)
xLeftHalf = sUtil.binarySearch(startBounds=leftBounds, **hwhmKwargs)
assert abs(xLeftHalf - (center - fwhm/2)) < assertionTolerance
assert abs(xRightHalf - (center + fwhm/2)) < assertionTolerance

### Non-monotonic but still well defined
There is only one value in the domain that satisfies. It starts off bracked

No test for when there is a peak in the middle and it starts *not* bracketed, 
i.e. if rightStart fwhm was 0.75

To handle this, bracketSearch would have to report that it bracketed on both sides

In [ ]:
rightStart = center + fwhm*.45
for leftStart in [0, center - fwhm, center - 0.55 * fwhm]:
    xLeftHalf = sUtil.binarySearch(startBounds=[leftStart, rightStart], **hwhmKwargs)
#     assert abs(xLeftHalf - (center - fwhm/2)) < assertionTolerance

### Bad bound conditioning that should succeed
But they are not very robust to noise

In [ ]:
# Starts outside, but pretty easy
xLeftHalf = sUtil.binarySearch(startBounds=[0, xPeak/4], **hwhmKwargs)
assert abs(xLeftHalf - (center - fwhm/2)) < assertionTolerance

# Very bad domain (This takes a long time)
# noise = 0
# xLeftHalf = sUtil.binarySearch(startBounds=[0, .02], **hwhmKwargs)
# assert abs(xLeftHalf - (center - fwhm/2)) < assertionTolerance

# Target very close to peak
noise = defaultNoise / 10 # turn down noise a little bit
for trialAgainstNoise in range(5):
    try:
        xRightOnPeak = sUtil.binarySearch(startBounds=[0, xPeak/4], targetY=0.99*amp, **binSearchOpts)
        break
    except RangeError as err:
        if 'probably noise' in err.args[0]:
            continue
        else:
            raise err
else:
    raise Exception('We tried a lot but noise killed this one')
assert abs(xRightOnPeak - center) < assertionTolerance
noise = defaultNoise


### These should generate errors

In [ ]:
# Targeting outside of hard constrain domain
try:
    sUtil.binarySearch(startBounds=[xPeak, xPeak+.1], targetY=0, hardConstrain=True, **binSearchOpts)
    assert False
except RangeError as err:
    assert err.args[1] == 'low'
    
# Targeting something too high
try:
    goodAsItGets = sUtil.binarySearch(startBounds=[0, center - 0.45 * fwhm], targetY=2, hardConstrain=False, **binSearchOpts)
    assert False
except RangeError as err:
    assert err.args[1] == 'high'